| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb)  | **Extensions:** | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |

# Working with Mineral Composition Data


### Mineral Database
pyrolite includes a limited mineral database which is useful for looking up endmember compositions.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pyrolite.mineral.mindb import (
    list_groups,
    list_minerals,
    list_formulae,
    get_mineral,
    get_mineral_group,
)

pd.options.display.precision = 3  # smaller graphical outputs

From the database, you can get the list of its contents using a few utility
functions:



In [ ]:
list_groups()

In [ ]:
", ".join(
    list_minerals()
)  # string the list of minerals together so we can print it succinctly

In [ ]:
", ".join(
    list_formulae()
)  # string the list of minerals formulae so we can print it succinctly

You can also directly get the composition of specific minerals by name. These will be a `pd.Series` of elemental Wt% data (although derived from the formuale directly), and normalised to 100%:

In [ ]:
get_mineral("forsterite")

If you want to get compositions for all minerals within a specific group, you can
use `get_mineral_group`:



In [ ]:
get_mineral_group("olivine")

In some cases, these might give some useful reference points for comparing your mineral or whole-rock data to:

In [ ]:
px_comps = get_mineral_group("pyroxene").pyrochem.convert_chemistry(
    to=["MgO", "SiO2", "CaO", "FeO", "MnO", "Al2O3", "Na2O"]
)
px_comps.iloc[:, 2:] = (
    px_comps.iloc[:, 2:].fillna(0) * 100
)  # pyroxene compositions as Wt% oxides
ax = px_comps[["CaO", "MgO"]].pyroplot.scatter(c="k", s=10)

If you wanted to, you could also label these (noting there's some overlap to later deal with..):

In [ ]:
ax = px_comps[["CaO", "MgO"]].pyroplot.scatter(c="k", s=10)
for ix, row in px_comps.iterrows():
    ax.annotate(
        row["name"], xy=(row[["CaO", "MgO"]]), xytext=(5, 5), textcoords="offset points"
    )

In [ ]:
get_mineral_group("pyroxene")

In [ ]:
px_comps[["CaO", "MgO", "Al2O3"]].pyroplot.scatter(c="k", marker="o", s=100, zorder=10)

### Mineral Endmember Decomposition

A common task when working with mineral chemistry data is to take measured compositions
and decompose these into relative proportions of mineral endmember compositions.
pyrolite includes some utilities to achieve this and a limited mineral database
for looking up endmember compositions.

In [ ]:
import pandas as pd
import numpy as np
from pyrolite.mineral.mindb import get_mineral
from pyrolite.mineral.normative import endmember_decompose

First we'll start with a composition of an unknown olivine:




In [ ]:
comp = pd.Series({"MgO": 42.06, "SiO2": 39.19, "FeO": 18.75})
df = pd.DataFrame(
    [comp, comp]
)  # a dataframe with this composition twice, to illustrate what you might do with a table of olivine compositions

We can break this down into olivine endmebmers using the
`endmember_decompose` function:

In [ ]:
ed = endmember_decompose(df, endmembers="olivine", order=1, molecular=True)
ed

Equally, if you knew the likely endmembers beforehand, you could specify a list of
endmembers:




In [ ]:
ed = endmember_decompose(
    df, endmembers=["forsterite", "fayalite"], order=1, molecular=True
)
ed

We can check this by recombining the components with these proportions. We can first
lookup the compositions for our endmembers:




In [ ]:
em = pd.DataFrame(
    [get_mineral("forsterite"), get_mineral("fayalite")]
)  # build a dataframe of our endmembers
em

In [ ]:
em.loc[:, ~(em == 0).all(axis=0)]  # get the columns where all the values are not zero

First we have to convert these element-based compositions to the oxide-based compositions we started with (or alternatively, we could go the other way around):

In [ ]:
emvalues = (
    em.loc[:, ["Mg", "Si", "Fe"]]
    .pyrochem.to_molecular()
    .fillna(0)
    .pyrochem.convert_chemistry(to=["MgO", "SiO2", "FeO"], molecular=True)
    .fillna(0)
    .pyrocomp.renormalise(scale=1)
)
emvalues

These can now be used with our endmember proportions to regenerate a composition, noting that these are molecular - we'll need to convert them to Wt%:


In [ ]:
recombined = pd.DataFrame(
    ed.values @ emvalues
).pyrochem.to_weight()  # multiply the endmember proportions by the endmember compositions to get a weighted composition in wt%
recombined

To make sure these compositions are within 0.01 percent:




In [ ]:
assert np.allclose(recombined.values, comp.values, rtol=10 ** -4)

----
**Optional:** Check out some of the other minerals and mineral groups.

In [ ]:
from pyrolite.mineral.mindb import get_mineral_group, list_groups, list_minerals

list_groups()

In [ ]:
get_mineral_group('olivine')

----
<div class='alert alert-warning'> <font size="+1" color="black"><b> Checkpoint & Time Check</b><br>How are things going?</font></div>

----

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb)  | **Extensions:** | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |